In [1]:
import torch
import numpy as np

from torch.utils.data import Dataset, DataLoader

from utils import normalize, onehot, count_parameters
from model import TransformerModel
from sEMGdata import sEMGData, sEMGDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Additional Info when using cuda
if device.type == 'cuda':
  print(f"Device: {torch.cuda.get_device_name(0)}")
  torch.cuda.empty_cache()

Device: NVIDIA GeForce RTX 3090


In [3]:
# load the data and labels
sEMG_vowels = sEMGData(file_dir="data/subjects_40_vowels_v6.mat")

# leave-one-subject-out data partition
x_train, y_train, x_test, y_test = sEMG_vowels.load_data(sub_test=0, sub_normalize=True)
# Normalize the signal per channel
x_train, x_test = normalize(x_train, x_test)
# Onehot encode the label
y_train, y_test = onehot(y_train), onehot(y_test)
# TODO: split into training and validation

train_dataset = sEMGDataset(x_train, y_train, dtype=np.float16)

Total number of subjects:       40
Testing Subject ID:             44
Testing Subject VFI-1:          0
# of Testing Samples:           165
# of Healthy Training Samples:  3037
# of Fatigued Training Samples: 3270


In [4]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
model = TransformerModel(feat_dim=4, d_model=64, n_head=8, d_hid=256, n_layer=1)
batcher = iter(train_loader)
model.half().to(device)
print(f"Total Model Parameters: {count_parameters(model)}")

x,y = next(batcher)
print(x.shape, x.dtype)
print(y.shape, y.dtype)

Total Model Parameters: 818306
torch.Size([32, 4000, 4]) torch.float16
torch.Size([32, 2]) torch.float16


In [5]:
x, y = x.to(device), y.to(device)
y_pred = model(x)
print(y_pred.shape, y_pred.dtype)

torch.Size([32, 2]) torch.float16


In [6]:
if device.type == 'cuda':
  print(torch.cuda.get_device_name(0))
  print('Memory Usage:')
  print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
  print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
  torch.cuda.empty_cache()

NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 0.4 GB
Cached:    0.4 GB


In [ ]:
import matplotlib.pyplot as plt
i = np.random.randint(x_train.shape[0])
plt.plot(x_train[i,:,0])
plt.xlim([0, 4000])
plt.ylim([-5,5])
if y_train[i,0] == 1:
  plt.title("Healthy Sample")
else:
  plt.title("Fatigued Sample")